In [1]:
from codemixeddata import *
import posfeatures as pos
import lidffeatures as lidf
from pipeline import *
from customutils import *
from sklearn_crfsuite import scorers, metrics, CRF
import numpy as np
from sklearn.cross_validation import cross_val_score
from sklearn.pipeline import Pipeline
import pandas as pd

### Shallow Parser accuracy with gold pos, gold normalization, gold language id.

In [11]:
X, y = LoadDataSet(xlabels=['WORD', 'LANG', 'NORM', 'POS'], ylabel='CHUNK')

In [12]:
results = cross_val_score(CRF(), X, y, scorers.flat_accuracy, cv=10, n_jobs=-1)

Accuracy {{'%.2f' % (results.mean() * 100)}}

## Language Identification

In [2]:
X, y = LoadDataSetWFeatures(xlabels=['WORD', 'POSITION', 'EPOS', 'EPOSSCORE'], ylabel='LANG')

In [3]:
features = FeatureStacker([ #('lex', lidf.AddLexTransformer()),
#                            ('position', lidf.AddPositionTransformer()),
                            ('bnc_count', lidf.BNCCountsTransformer()),
                            ('lex_norm', lidf.LexNormCountsTransformer()),
                            ('in_bnc_or_lexnorm', lidf.LexNormBNCSpecialTransformer()),
                            ('hindi_dict', lidf.HindiDictionaryCountsTransformer()),
#                            ('word_length', lidf.WordLengthTransformer()),
                            ('capitalization', lidf.CapitalizationTransformer()),
                            ('cngram', lidf.CharacterNgramTransformer()),
                            ('en_rest_pos', lidf.PoSTransformer()),
                            ('en_rest_pos_confidence', lidf.PoSConfidenceTransformer()),
                            ('affixes', pos.AffixesTransformer())
                          ])

model = Pipeline([
        ('features', features),
        ('classifier', CRF())
    ])

In [4]:
lidf_acc = cross_val_score(model, X, y, scorers.flat_accuracy, cv=10, n_jobs=-1)

Accuracy {{'%.2f' % (lidf_acc.mean() * 100)}}

## POS tagger

In [2]:
X, y = LoadDataSetWFeatures(xlabels=['WORD', 'POSITION', 'EPOS', 'EPOSSCORE', 'LANG', 'WX'], ylabel='POS')

In [4]:
X = run_step(X, None, 'WX')

In [7]:
features = FeatureStacker([ ('lex', pos.AddLexTransformer()),
                            ('gold_lang', pos.GoldLangTransformer()),
                            ('en_rest_pos', pos.PoSTransformer()),
                            ('en_rest_pos_confidence', pos.PoSConfidenceTransformer()), 
                            ('affixes', pos.AffixesTransformer()),
                            ('en_clusters', pos.HWCTransformer()),
                          ])

model = Pipeline([
        ('features', features),
        ('classifier', CRF())
    ])
model.set_params(features__affixes__form=1)

Pipeline(steps=[('features', FeatureStacker(transformer_list=[('lex', AddLexTransformer()), ('gold_lang', GoldLangTransformer()), ('en_rest_pos', PoSTransformer(ignore=False)), ('en_rest_pos_confidence', PoSConfidenceTransformer(ignore=False)), ('affixes', AffixesTransformer(form=1, prefix_len=5, suffix_len=...
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False))])

In [8]:
pos_acc = cross_val_score(model, X, y, scorers.flat_accuracy, cv=10, n_jobs=-1)

Accuracy {{'%.2f' % (pos_acc.mean() * 100)}}